In [ ]:
import pandas as pd

In [ ]:
data_eng=pd.read_csv('/content/English_final.csv')

In [ ]:
data_eng['Id']=data_eng['Unnamed: 0']
data_eng['Cleaned_tweet']=data_eng['Tweet']
data_eng=data_eng.drop(['Unnamed: 0','Tweet'],axis=1)

In [ ]:
#data_res=pd.read_csv('/content/Results.csv')
#data_res=data_res.drop(['Unnamed: 0'],axis=1)
#pd.concat([data_eng,data_res],axis=1).dropna()

In [ ]:
import spacy
import nltk

In [ ]:
nlp=spacy.load('en_core_web_sm')

In [ ]:
def show_ents(doc): 
  lis=[]
  if doc.ents: 
    for ent in doc.ents: 
      if(ent.label_=='ORG' or ent.label_=='PRODUCT'):
        if(ent.text.lower() not in lis):
          lis.append(ent.text.lower())
  return(lis)

In [ ]:
aspect_lis=[]
for i in range(len(data_eng)):
  aspect_lis+=show_ents(nlp(data_eng['Cleaned_tweet'][i]))

In [ ]:
#Unique aspects
aspect_entity_lis=[]
for word in aspect_lis:
  if(word not in aspect_entity_lis):
    aspect_entity_lis.append(word)

In [ ]:
def pos_tagging(data):
    #print("pos tagging")
    req_tag = ['NN','NNP','NNS']
    extracted_words = []
    i = 0
    try:
        for x in data['Cleaned_tweet'].values:
            doc =nlp(x)
            for token in doc:
                i += 1
                if token.tag_ in req_tag and token.shape_ != 'x' and token.shape_ != 'xx' and token.shape_ != 'xxx':
                  if(token.lemma_ not in extracted_words):
                    extracted_words.append(token.lemma_)
        return extracted_words
    except Exception as e:
        return extracted_words
      

In [ ]:
#Collecting all the Nouns
extract_words = pos_tagging(data_eng)

In [ ]:
from gensim.models import word2vec

In [ ]:
#!pip install nltk
#!pip install gensim

In [ ]:
corpus=[]
for i in range(len(data_eng)):
  lis=[]
  for word in data_eng['Cleaned_tweet'][i].split():
    lis.append(word)
  corpus.append(lis)  

In [ ]:
#corpus=data_eng['Cleaned_tweet'].values

In [ ]:
import gensim 
from gensim.models import Word2Vec
model_wiki=gensim.models.Word2Vec(corpus, size = 300)

In [ ]:
def word2vec(corpus):
    terms = corpus
    try:
      # removal of words which not present in the word2vec model vocabulary. (wrongly spelled)
      filtered_terms = []
      for i in range(len(terms)):
        for word in terms[i]:
          filtered_terms.append(word) 
        

        
        #converting words into vector
      vector_of_terms = []
      for x in range(len(filtered_terms)):
        vector_of_terms.append(model_wiki.wv[filtered_terms[x]])
      return vector_of_terms,filtered_terms
    
    except Exception as e:
      return abort(Response(
            json.dumps({'status_code': 400, 'success': False, 'message': 'Something went wrong'}),
            mimetype="application/json"))

In [ ]:
vector_of_terms = []
for x in range(len(vocab)):
  vector_of_terms.append(model_wiki.wv[vocab[x]])
      

In [ ]:
import pandas as pd

afinn_wl_url = ('https://raw.githubusercontent.com'
                '/fnielsen/afinn/master/afinn/data/AFINN-111.txt')

afinn_wl_df = pd.read_csv(afinn_wl_url,
                          header=None, # no column names
                          sep='\t',  # tab sepeated
                          names=['term', 'value']) #new column names

seed = 808 # seed for sample so results are stable

In [ ]:
pos=[]
neg=[]
for i in range(len(afinn_wl_df)):
  if(afinn_wl_df['value'][i]>=0):
    pos.append(afinn_wl_df['term'][i])
  else:
    neg.append(afinn_wl_df['term'][i])  

In [ ]:
#afinn_wl_df['value'].value_counts()

In [ ]:
# create a list of globally defined positive and negative words to identify sentiment
# sentiment score based on the laxicon neg, pos words
def feature_sentiment(sentence, pos, neg):
    '''
    input: dictionary and sentence
    function: appends dictionary with new features if the feature
              did not exist previously,then updates sentiment to
              each of the new or existing features
    output: updated dictionary
    '''
    sent_dict = dict()
    sentence = nlp(sentence)
    opinion_words = neg + pos
    debug = 0
    for token in sentence:
        #sentiment=0.01
        #print(token.head)
        # check if the word is an opinion word, then assign sentiment
        if token.text in opinion_words:
            sentiment = 1 if token.text in pos else -1
            # if target is an adverb modifier (i.e. pretty, highly, etc.)
            # but happens to be an opinion word, ignore and pass
            if (token.dep_ == "advmod"):
              continue
            elif (token.dep_ == "amod"):
              sent_dict[token.head.text] = sentiment
          # for opinion words that are adjectives, adverbs, verbs...
            else:
              for child in token.children:
                # if there's a adj modifier (i.e. very, pretty, etc.) add more weight to sentiment
                # This could be better updated for modifiers that either positively or negatively emphasize
                if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                  sentiment *= 1.5
                  # check for negation words and flip the sign of sentiment
                if child.dep_ == "neg":
                  sentiment *= -1
              for child in token.children:
                # if verb, check if there's a direct object
                if (token.pos_ == "VERB") & (child.dep_ == "dobj"):                        
                  sent_dict[child.text] = sentiment
                  # check for conjugates (a AND b), then add both to dictionary
                  subchildren = []
                  conj = 0
                  for subchild in child.children:
                    if subchild.text == "and":
                      conj=1
                    if (conj == 1) and (subchild.text != "and"):
                      subchildren.append(subchild.text)
                      conj = 0
                  for subchild in subchildren:
                    sent_dict[subchild] = sentiment

                # check for negation
              for child in token.head.children:
                noun = ""
                if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                  sentiment *= 1.5
                  # check for negation words and flip the sign of sentiment
                if (child.dep_ == "neg"): 
                  sentiment *= -1
                
                # check for nouns
              for child in token.head.children:
                noun = ""
                if (child.pos_ == "NOUN") and (child.text not in sent_dict):
                  noun = child.text
                  # Check for compound nouns
                  for subchild in child.children:
                    if subchild.dep_ == "compound":
                      noun = subchild.text + " " + noun
                    sent_dict[noun] = sentiment
                    debug += 1
    return sent_dict

In [ ]:
# example 
tweet = data_eng['Cleaned_tweet'][1]
print (feature_sentiment(tweet, pos, neg))
## Output: {'food': 1, 'service': -1}

{'cameras': 1}


In [ ]:
data_eng['Cleaned_tweet'][1]

'Want hands GalaxyS21 pleadingface Samsung GalaxyS21 Ultra 5G premium Smartphone yet strong focus cameras'